# AutoGen Function Calling

This tutorial will redo the day-3-function-calling-with-llamaIndex.ipynb tutorial using AutoGen.

The tutorial demonstrates how to use AutoGen to create agents that translate natural language queries into SQL. We'll focus on:

1. Designing effective agents for SQL translation
2. Setting up proper agent interactions
3. Using function calling to execute database operations
4. Creating natural conversations about data

First install the autogen-agentchat package.
```
%pip install autogen-agentchat~=0.2
```

## Basic Setup

First, set up the llm connection and sql connection.

In [1]:
import os
from dotenv import load_dotenv
import sqlite3

load_dotenv()

llm_list = [
    {
        "model": "gpt-4o",
        "api_key": os.getenv("AZURE_OPENAI_KEY"),
        "api_version": os.getenv("OPENAI_API_VERSION"),
        "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
    }
]

llm_config = {
    "temperature": 0.7,
    "config_list": llm_list,
    "timeout": 60,
}

# set up the sql connection
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

# Understanding AutoGen's Assist Agent and User Proxy

## Core Concepts

AutoGen provides a framework for building conversational AI agents that can interact with each other and with users. Two fundamental agent types in AutoGen are:

1. **AssistantAgent (Assist Agent)**
   - An AI-powered agent that can understand and respond to queries
   - Processes information and generates responses using an LLM
   - Can use tools/functions when configured
   - Maintains conversation context
   - Typically plays the role of the "expert" or "assistant"

2. **UserProxyAgent (User Proxy)**
   - Acts as an intermediary between human users and assistant agents
   - Can execute code and functions
   - Manages user interactions
   - Can be configured to run autonomously or require human approval
   - Helps maintain conversation flow and state

## Implementation Example

Let's create a basic example showing how these agents interact:

```python
import autogen

# Configure the assistant
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "temperature": 0.7,
        "config_list": [{
            "model": "gpt-4",
            "api_key": "your_api_key",
            "base_url": "your api endpoint"
        }],
    },
    system_message="You are a helpful AI assistant that provides clear and concise answers."
)

# Configure the user proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",  # TERMINATE, NEVER, or ALWAYS
    max_consecutive_auto_reply=2,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding", # directory for code execution
        "use_docker": False, 
    },
)
```

### Key Features

#### AssistantAgent Features:
- **System Message**: Defines the agent's role and behavior
- **LLM Configuration**: Specifies the language model and parameters
- **Function Calling**: Can be configured with custom functions
- **Memory**: Maintains conversation history

#### UserProxyAgent Features:
- **Human Input Modes**:
  - `TERMINATE`: Asks for human input only when terminating
  - `NEVER`: Runs completely autonomously
  - `ALWAYS`: Always asks for human input
- **Code Execution**: Can execute code in a controlled environment
- **Auto-reply Settings**: Controls consecutive automatic responses
- **Termination Conditions**: Customizable conversation ending criteria


## Agent Design

In this section, we will implement the AutoGen function calling feature, which performs the same tasks as described in [function calling with LlamaIndex](day-3-function-calling-with-llamaIndex.ipynb).

In [ ]:
from autogen import (
    AssistantAgent,
    UserProxyAgent
)